## Follow-Up using `soynlp`

- `soynlp`를 이용, 띄어쓰기 교정 한 후 동일한 모델에 학습시키기 위함
- 띄어쓰기 학습 데이터로 4,030건의 뉴스 기사 22,039 문장을 사용했으나, 리뷰 교정엔 적용하기 어려웠음 (영화 리뷰에서 학습용 데이터 선정 요망)

In [1]:
from pprint import pprint
from soyspacing.countbase import RuleDict, CountSpace
import soyspacing

from words_preprocessing import *
from file_io import *

In [2]:
train = load_pickle('../train_data.pkl')
test = load_pickle('../test_data.pkl')

In [3]:
corpus_fname = '../134963_norm.txt'
model = CountSpace()
model.train(corpus_fname)

all tags length = 654328 --> 53317, (num_doc = 15598)

In [4]:
model_fname = '../model/spacing.model'
model.save_model(model_fname, json_format=False)

model = CountSpace()
model.load_model('../model/spacing.model', json_format=False)

### 띄어쓰기 교정 함수 적용

- arguments
    - 4개의 parameter
        - `force_abs_threshold` : 점수의 절대값이 이 수준 이상이면 최고점이 아니더라도 즉각 태깅
        - `nonspace_threshold` : 이 점수 이하일 때만 0으로 태깅
        - `space_threshold` : 이 점수 이상일 때만 1로 태깅
        - `min_count` : L, C, R 각각의 feature 빈도수가 min_count 이하이면 불확실한 정보로 판단, 띄어쓰기 계산 시 무시
        - `verbose`: iteration 마다 띄어쓰기가 어떻게 되고 있는지 확인

rules : 점수와 관계없이 반드시 태깅을 먼저 할 (chars, tags)

In [5]:
verbose=False
mc = 10  # min_count
ft = 0.3 # force_abs_threshold
nt =-0.3 # nonspace_threshold
st = 0.3 # space_threshold

In [6]:
N = 24

pprint(train[N][0][0])
sent_corrected, tags = model.correct(doc=train[N][0][0],
                                     verbose=verbose,
                                     force_abs_threshold=ft,
                                     nonspace_threshold=nt,
                                     space_threshold=st,
                                     min_count=mc)

pprint(sent_corrected)

'고생하셨습니다. 감사합니다.'
'고생하셨습니다. 감사합니다.'


In [7]:
%%time
train_spaced = [(model.correct(row[0][0], 
                               verbose=verbose,
                               force_abs_threshold=ft,
                               nonspace_threshold=nt,
                               space_threshold=st,
                               min_count=mc)[0],
                 row[1]) for row in train]

CPU times: user 1min 43s, sys: 192 ms, total: 1min 43s
Wall time: 1min 43s


In [8]:
test_spaced = [(model.correct(row[0][0], 
                              verbose=verbose,
                              force_abs_threshold=ft,
                              nonspace_threshold=nt,
                              space_threshold=st,
                              min_count=mc)[0],
                 row[1]) for row in test]

In [9]:
pprint(train_spaced[20])
pprint(test_spaced[20])

('재밌다! 에로영화가 나아가야 할 방향을 제시해주는 작품이랄까', 0)
('나는 나에게 작은 손을 내밀어 눈물과 위안으로 잡는 최초의 악수. 자존. 이해. 화해. 성장담이 아름다운 이유.', 3)


In [10]:
from soynlp.word import WordExtractor

word_extractor = WordExtractor(min_count=10,
                               min_cohesion_forward=0.05, 
                               min_right_branching_entropy=0.0)

sentences_spaced = [row[0] for row in train_spaced]
word_extractor.train(sentences_spaced) # list of str or like
words = word_extractor.extract()

training was done. used memory 0.990 Gbse memory 0.903 Gb
all cohesion probabilities was computed. # words = 49621
all branching entropies was computed # words = 117573
all accessor variety was computed # words = 117573


In [11]:
from soynlp.tokenizer import MaxScoreTokenizer

scores = {items[0]: items[1][0] for items in list(words.items())}
tokenizer = MaxScoreTokenizer(scores=scores)

train_tokenized = [(tokenizer.tokenize(row[0][0]), row[1]) for row in train]
test_tokenized = [(tokenizer.tokenize(row[0][0]), row[1]) for row in test]
train_tokenized[N]

(['고생하셨습니다.', '감사', '합니다', '.'], 0)

In [13]:
save_pickle('../train_space_tokenized.pkl' , train_tokenized)
save_pickle('../test_space_tokenized.pkl' , test_tokenized)
save_pickle('../train_space_corrected.pkl', train_spaced)
save_pickle('../test_space_corrected.pkl', test_spaced)